# Parameter Management

In [1]:
import torch 
import torch.nn as nn

net = nn.Sequential(nn.Linear(20, 256),
                    nn.ReLU(),
                    nn.Linear(256, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        # Initialize weight parameter by a normal distribition 
        # with a mean of 0 and standard deviation of 0.01.
        nn.init.normal_(m.weight.data, std=0.01)
        # The bias parameter is initialized to zero by default.
        m.bias.data.fill_(0.0)

net.apply(init_weights)
x = torch.randn(2,20) 
net(x)           

tensor([[-0.0015, -0.0002,  0.0014, -0.0040, -0.0015, -0.0003,  0.0035,  0.0027,
         -0.0072, -0.0021],
        [ 0.0019,  0.0021, -0.0010,  0.0040,  0.0029, -0.0029,  0.0023,  0.0018,
         -0.0007, -0.0057]], grad_fn=<AddmmBackward>)

## Parameter Access

In [2]:
print('Netwrork structure: ', net)
print(net[0].parameters)
print(net[1].parameters)
print(net[2].parameters)

Netwrork structure:  Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)
<bound method Module.parameters of Linear(in_features=20, out_features=256, bias=True)>
<bound method Module.parameters of ReLU()>
<bound method Module.parameters of Linear(in_features=256, out_features=10, bias=True)>


### Targeted Parameters

In [3]:
print(net[0].weight)
print(net[0].bias)

Parameter containing:
tensor([[-2.1886e-03,  1.3199e-02, -1.3012e-03,  ..., -4.3397e-04,
          7.1551e-03,  3.5285e-03],
        [ 3.4181e-03,  3.9722e-04, -1.2453e-02,  ...,  8.6626e-03,
          1.0587e-02, -6.4080e-03],
        [ 8.8158e-03, -8.6248e-03, -7.9011e-03,  ...,  4.7793e-03,
          5.7793e-03,  1.4681e-02],
        ...,
        [-6.3947e-03,  8.4270e-05, -1.9869e-03,  ...,  1.2666e-03,
          9.4543e-03, -5.2006e-03],
        [ 1.9524e-03,  6.6059e-03,  1.9889e-02,  ...,  4.1482e-03,
          1.0078e-02, -2.2610e-02],
        [ 5.9135e-03,  1.4009e-02,  8.0821e-04,  ..., -4.6321e-03,
         -6.7223e-04, -7.1269e-03]], requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

### Access Gradient

As we have not invoke backprobagation, the output is None

In [4]:
net[0].weight.grad

### All Parameters at Once

In [5]:
print(net.state_dict)

<bound method Module.state_dict of Sequential(
  (0): Linear(in_features=20, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)>


## Parameter Initialization

### Gaussian random Initialization

In [6]:
# force_reinit ensures that the variables are initialized again
torch.nn.init.normal_(net[0].weight, mean=0, std =0.01)
net[0].weight.data[0]

tensor([-0.0029, -0.0009,  0.0035,  0.0085,  0.0021, -0.0038,  0.0003,  0.0065,
        -0.0123, -0.0097, -0.0108,  0.0028, -0.0066,  0.0122, -0.0136,  0.0072,
        -0.0075,  0.0047, -0.0053, -0.0112])

### Constant Initialization 

In [7]:
torch.nn.init.constant_(net[0].weight, val=1)
net[0].weight.data[0]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1.])

### Multiple Initializations 

In [8]:
torch.nn.init.constant_(net[2].weight, val=42)
torch.nn.init.xavier_uniform_(net[0].weight)
print(net[2].weight.data[0,0])
print(net[0].weight.data[0])

tensor(42.)
tensor([ 0.0791, -0.1439, -0.0874, -0.0939,  0.0020,  0.1474, -0.1367,  0.0813,
         0.0922, -0.0469, -0.0855,  0.0612, -0.0373,  0.0666, -0.1339,  0.0973,
         0.0735, -0.0855, -0.0604,  0.0672])


### Custom Initialization

$$
\begin{aligned}
    w \sim \begin{cases}
        U[5, 10] & \text{ with probability } \frac{1}{4} \\
            0    & \text{ with probability } \frac{1}{2} \\
        U[-10, -5] & \text{ with probability } \frac{1}{4}
    \end{cases}
\end{aligned}
$$

In [9]:
# class MyInit(init.Initializer):
#     def _init_weight(self, name, data):
#         print('Init', name, data.shape)
#         data[:] = nd.random.uniform(low=-10, high=10, shape=data.shape)
#         data *= data.abs() >= 5
def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5
net.apply(init_weights)
net[0].weight.data[0]

tensor([-8.1520, -0.0000, -0.0000, -7.0293, -0.0000, -8.3091,  8.7162,  0.0000,
         8.7612, -0.0000,  0.0000, -9.2657, -0.0000,  5.0226, -0.0000,  0.0000,
        -0.0000,  8.8486,  5.5990, -6.6388])

### Modify Parameters Directly 

In [10]:
net[0].weight.data[:] += 1
net[0].weight.data[0,0] = 42
net[0].weight.data[0]

tensor([42.0000,  1.0000,  1.0000, -6.0293,  1.0000, -7.3091,  9.7162,  1.0000,
         9.7612,  1.0000,  1.0000, -8.2657,  1.0000,  6.0226,  1.0000,  1.0000,
         1.0000,  9.8486,  6.5990, -5.6388])

## Tied Parameters


In [11]:
shared = nn.Sequential(nn.Linear(8,8),
                    nn.ReLU()
                    )
tied_net = nn.Sequential(nn.Linear(20, 8),
                        nn.ReLU(),
                        *shared,
                        *shared,
                        nn.Linear(8, 10))

def init_weights(m):
    if type(m) == nn.Linear:
        torch.nn.init.xavier_uniform_(m.weight)

tied_net.apply(init_weights)
tied_net(x)

print(tied_net)

# Check whether the parameters are the same
print(tied_net[2].weight.data[0] == tied_net[4].weight.data[0])
tied_net[2].weight.data[0,0] = 100
# And make sure that they're actually the same object rather than just having the same value.
print(tied_net[2].weight.data[0] == tied_net[4].weight.data[0])

Sequential(
  (0): Linear(in_features=20, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=8, bias=True)
  (3): ReLU()
  (4): Linear(in_features=8, out_features=8, bias=True)
  (5): ReLU()
  (6): Linear(in_features=8, out_features=10, bias=True)
)
tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])
